In [124]:
# !pip install -q torchdata torchtext spacy altair GPUtil
# !python -m spacy download de_core_news_sm
# !python -m spacy download en_core_web_sm

In [125]:
# !pip install torchtext==0.16.0
# !pip install -q torchdata spacy altair GPUtil

In [126]:
import os
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax,pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
# import torchtext
# from torchtext.data.functional import to_map_style_dataset
# from torch.utils.data import DataLoader
# from torchtext.vocab import build_vocab_from_iterator
# import torchtext.datasets as datasets
# import spacy
# import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP

warnings.filterwarnings("ignore")
RUN_EXAMPLES = True
print(torch.__version__)
# print(torchtext.__version__)

2.6.0+cu124


#Helper functions:

In [127]:
def is_interactive_notebook():
    return __name__ == "__main__"


def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)


def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)


class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None


class DummyScheduler:
    def step(self):
        None

# This is the main model class:

In [128]:
class EncoderDecoder(nn.Module):

  """
    Main Architecture of the encoder and decoder
  """

  def __init__(self, encoder, decoder, source_embeddings, target_embeddings, generator):
    super(EncoderDecoder, self).__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.generator = generator
    self.source_embeddings = source_embeddings
    self.target_embeddings = target_embeddings

  def forward():
    pass

  def encode(self, source, source_mask):
    return self.encoder(self.source_embeddings(source),source_mask)

  def decode(self, memory, source_mask, target, target_mask):
    return self.decoder(self.target_embeddings(target), memory, source_mask, target_mask)

# Now we build the encoder, decoder, self-attention, generator, embedding classes.

In [146]:
class Generator(nn.Module):
  def __init__(self, d_model, vocab) -> None:
     super(Generator,self).__init__()
     self.projections = nn.Linear(d_model, vocab)

  def forward(self,x):
    return log_softmax(self.projections(x), dim=1)

In [130]:
def clones(module, N):
  """
  Duplicating layers

  """
  return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

#Encoder Layer

In [131]:
class Encoder(nn.Module):
  "Stack of N layers"

  def __init__(self, layer, N):
    super(Encoder,self).__init__()
    self.layers = clones(layer,N)
    self.norm = LayerNorm(layer.size)

  def forward(self,x, mask):
    "pass the input and mask thu each layer"
    for layer in self.layers:
      x  = layer(x,mask)
    return self.norm(x)

In [132]:
class LayerNorm(nn.Module):
    "Construct a layernorm module as per paperand standards for reducing training time "

    def __init__(self, features, eps = 1e-6):
      super(LayerNorm, self).__init__()
      self.a_2 = nn.Parameter(torch.ones(features))
      self.b_2 = nn.Parameter(torch.zeros(features))
      self.eps = eps

    def forward(self, x):
      mean = x.mean(-1, keepdim=True)
      std = x.std(-1, keepdim=True)
      return self.a_2 * (x - mean)/ (std + self.eps) + self.b_2


In [133]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    for code simplicity the norm is first as opposed to last.
    """

    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

In [134]:
class EncoderLayer(nn.Module):
  "Self attention layer and feed forward Layer"
  def __init__(self, size, self_attn, feed_forward, dropout):
    super(EncoderLayer,self).__init__()
    self.self_attn = self_attn
    self.feed_forward = feed_forward
    self.sublayer = clones(SublayerConnection(size, dropout), 2)
    self.size = size

  def forward(self,x, mask):
    x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
    return self.sublayer[1](x, self.feed_forward)


#Decoder Layer

In [135]:
class Decoder(nn.Module):
  "N layer Decoder"
  def __init__(self,layer, N):
    super(Decoder, self).__init__()
    self.layers = clones(layer,N)
    self.norm = LayerNorm(layer.size)

  def forward(self,x, memory, source_mask, target_mask):
    for layer in self.layers:
      x = layer(x, memory, source_mask, target_mask)
    return self.norm(x)

In [136]:
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"

    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [137]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(torch.uint8)
    return subsequent_mask == 0

# Attention mechanism:
Here i implement standard attention mechanism which is a scaled dot product attention.

In [138]:
def attention(query, key, value, mask=None, dropout=None):
  """Computing the scaled dot product"""
  dim_k = query.size(-1)
  scores = query @ key.transpose(-2,-1) / math.sqrt(dim_k)
  if mask is not None:
    scores = scores.masked_fill(mask == 0, -1e9)
  p_attn = scores.softmax(dim=-1)
  if dropout is not None:
    p_attn = dropout(p_attn)
  return p_attn @ value , p_attn

In [139]:
from re import X
class MultiHeadedAttention(nn.Module):
  def __init__(self, h, dim_model, dropout=0.1):
    "Take in model size and number of heads."
    super(MultiHeadedAttention, self).__init__()
    assert dim_model % h == 0
    # Assume dim of value and key are same
    self.dim_k = dim_model // h
    self.h = h
    self.linears = clones(nn.Linear(dim_model, dim_model), 4)
    self.attn = None
    self.dropout = nn.Dropout(p=dropout)

  def forward(self,query, key, value, mask = None):
    if mask is not None:
      mask = mask.unsqueeze(1)

    nbatches = query.size(0)

    query, key, value = [lin(x).view(nbatches, -1, self.h, self.dim_k).transpose(1,2) for lin, x in zip(self.linears,(query, key, value))]

    # Applying attention on all the projected vectors in batch
    x, self.attn = attention(query, key, value, mask=mask, dropout = self.dropout)

    # Concat using view and applying final linear
    x = (x.transpose(1,2).contiguous().view(nbatches,-1, self.h * self.dim_k))

    del query
    del key
    del value
    return self.linears[-1](x)


# Embeddings and positional encoding


In [140]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."

    def __init__(self, dim_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(dim_model, d_ff)
        self.w_2 = nn.Linear(d_ff, dim_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(self.w_1(x).relu()))

In [141]:
class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [142]:
class PositionalEncoding(nn.Module):
    "Implementing the PE function using sin and cos waves."

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

# Model:

In [143]:
def build_model(source_vocab, target_vocab, N=6, dim_model=512, dim_feed_forward=2048, h=8, dropout=0.1):
  """Constructing a model with the given parameters"""

  c = copy.deepcopy
  attn = MultiHeadedAttention(h,dim_model)
  ff = PositionwiseFeedForward(dim_model, dim_feed_forward, dropout)
  position = PositionalEncoding(dim_model, dropout)

  model = EncoderDecoder(
      Encoder(EncoderLayer(dim_model, c(attn), c(ff), dropout), N),
      Decoder(DecoderLayer(dim_model, c(attn), c(attn), c(ff), dropout), N),
      nn.Sequential(Embeddings(dim_model, source_vocab), c(position)),
      nn.Sequential(Embeddings(dim_model, target_vocab), c(position)),
      Generator(dim_model, target_vocab)
  )

  for p in model.parameters():
    if p.dim() > 1:
      nn.init.xavier_uniform_(p)
  return model

# Inferencing without training the transformer model

In [144]:
def inference_test():
  test_model = build_model(11,11,2)
  test_model.eval()
  source = torch.LongTensor([[1,2,3,4,5,6,7,8,9,10]])
  source_mask = torch.ones(1,1,10)

  memory = test_model.encode(source,source_mask)
  ys = torch.zeros(1,1).type_as(source)


  for i in range(9):
    out = test_model.decode(
        memory,source_mask, ys, subsequent_mask(ys.size(1)).type_as(source.data)
        )

    prob = test_model.generator(out[:, -1])
    _, next_word = torch.max(prob, dim=1)
    next_word = next_word.data[0]
    ys = torch.cat(
        [ys, torch.empty(1,1).type_as(source.data).fill_(next_word)], dim=1
    )

  print("Untrained model predictions: ", ys)


In [147]:
def run_tests():
  for _ in range(10):
    inference_test()

show_example(run_tests)

Untrained model predictions:  tensor([[0, 1, 7, 0, 1, 4, 4, 4, 4, 4]])
Untrained model predictions:  tensor([[0, 4, 1, 1, 1, 8, 1, 8, 1, 8]])
Untrained model predictions:  tensor([[ 0,  6, 10,  4,  5, 10,  1,  5, 10,  5]])
Untrained model predictions:  tensor([[0, 1, 8, 8, 8, 8, 8, 8, 8, 8]])
Untrained model predictions:  tensor([[ 0, 10,  4,  0, 10,  2,  2,  2,  2,  2]])
Untrained model predictions:  tensor([[0, 9, 5, 6, 0, 9, 5, 6, 0, 6]])
Untrained model predictions:  tensor([[ 0,  4, 10,  4, 10,  4, 10,  4, 10,  4]])
Untrained model predictions:  tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
Untrained model predictions:  tensor([[0, 9, 1, 1, 1, 1, 1, 1, 2, 2]])
Untrained model predictions:  tensor([[ 0,  9,  6,  2,  3, 10,  5,  4,  6,  2]])
